<a href="https://colab.research.google.com/github/AgomferAustral/DMA-Caras/blob/main/deteccionprod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Recorte de caras en las imagenes para producción
###Version 1.0
##Fotos originales en: Origen
##Fotos recortadas en: Detectadasprod

In [ ]:
import os

if not os.path.exists('/content/drive/MyDrive'):
    from google.colab import drive
    drive.mount('/content/drive')

## Cargar las librerias que vamos a utilizar


In [ ]:
import sys
import subprocess


def instalar_si_falta(paquete, nombre_import=None):
    if nombre_import is None:
        nombre_import = paquete
    try:
        __import__(nombre_import)
    except ImportError:
        print(f"Instalando {paquete}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", paquete])

# Lista de paquetes a instalar/importar
paquetes = [
    ("cv2", "opencv-python"),
    ("numpy", "numpy"),
    ("PIL", "Pillow")
]

for nombre_import, paquete in paquetes:
    instalar_si_falta(paquete, nombre_import)

# Ahora puedes importar normalmente
import cv2
import numpy as np
from PIL import Image


# Detectamos caras en imagenes
## La carpeta de origen se llama fotos
## y las guardamos en Detectadas

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Definicion de ruta de conexion - Origen - Destino imagenes

img_size=(120,120)

ruta_entrada = '/content/drive/MyDrive/fotos'
ruta_salida = f'/content/drive/MyDrive/Detectadasprod'


# prompt: verificar si la carpeta en "ruta_salida" existe. Si no existe, crearla

if not os.path.exists(ruta_salida):
  os.makedirs(ruta_salida)

def procesar_archivos_en_carpetas(ruta_principal,ruta_final, imgsize):
  """Recorre las carpetas dentro de la ruta principal y procesa los archivos.

  Args:
    ruta_principal: La ruta de la carpeta principal.
  """


  x_inicial = 0
  y_inicial = 0
  alto = imgsize[1]
  ancho = imgsize[0]
  img_size = (ancho ,alto)



  print(f"Ruta principal: {ruta_principal}")
  for carpeta_actual, _, archivos in os.walk(ruta_principal):
    print(f"Carpeta actual: {carpeta_actual}")
    for archivo in archivos:
      print(f"Archivo: {archivo}")
      ruta_completa = os.path.join(carpeta_actual, archivo)
      try:

            img = cv2.imread(ruta_completa, cv2.IMREAD_GRAYSCALE)

            img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

            if img is None:
                print(f"⚠️ No se pudo leer: {ruta_completa}")
                continue

            # Detectar rostros
            faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=10, minSize=(110, 110))

            for i, (x, y, w, h) in enumerate(faces):
                face = img[y:y+h, x:x+w]  # Recortar rostro
                face_resized = cv2.resize(face, img_size)  # Redimensionar

                face_cutted = face_resized[y_inicial:y_inicial+alto, x_inicial:x_inicial+ancho]

                # Crear carpeta de salida manteniendo la estructura original
                relative_path = os.path.relpath(carpeta_actual, ruta_entrada)
                output_folder = os.path.join(ruta_salida, relative_path)
                os.makedirs(output_folder, exist_ok=True)

                # Guardar rostro detectado
                output_path = os.path.join(output_folder, f"{os.path.splitext(archivo)[0]}_face{i}.jpg")
                cv2.imwrite(output_path, face_cutted)
                print(f"✅ Guardado: {output_path}")


      except Exception as e:
          print(f"❌ Error procesando {ruta_completa}: {e}")


      except Exception as e:
        print(f"Error al procesar el archivo {ruta_completa}: {e}")

# Deteccion de caras (escala de grises, recortes y escalado)
print("Detectando Caras ...")
procesar_archivos_en_carpetas(ruta_entrada, ruta_salida, img_size)
print("Detección inicial terminada ...")
print("Para entrenamiento, hay que limpiar manualmente la carpeta de salida, para eliminar los recortes que no son rostros.")

In [ ]:
# Verificar si la carpeta de salida existe, si no, crearla
img_size = (80, 80)
ruta_entrada = '/content/drive/MyDrive/Detectadasprod'
ruta_salida = '/content/drive/MyDrive/Detectadasprod2'

if not os.path.exists(ruta_salida):
    os.makedirs(ruta_salida)

# Inicializar el clasificador de rostros de OpenCV
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def procesar_archivos_en_carpetas(ruta_principal, ruta_final, imgsize):
    """
    Recorre las carpetas dentro de la ruta principal y procesa los archivos.
    Args:
        ruta_principal: La ruta de la carpeta principal.
        ruta_final: La ruta de la carpeta de salida.
        imgsize: Tupla con el tamaño de la imagen de salida.
    """
    fraccion = 8
    x_inicial = imgsize[0] // (fraccion - 2)
    y_inicial = imgsize[1] // (fraccion - 2)
    alto = imgsize[1]
    ancho = imgsize[0]
    multip = fraccion / (fraccion - 2)
    img_size = (math.floor(ancho * multip), math.floor(alto * multip))

    print(f"Ruta principal: {ruta_principal}")
    for carpeta_actual, _, archivos in os.walk(ruta_principal):
        print(f"Carpeta actual: {carpeta_actual}")
        for archivo in archivos:
            print(f"Archivo: {archivo}")
            ruta_completa = os.path.join(carpeta_actual, archivo)
            try:
                img = cv2.imread(ruta_completa, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"⚠️ No se pudo leer: {ruta_completa}")
                    continue

                img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

                # Detectar rostros
                faces = face_cascade.detectMultiScale(
                    img,
                    scaleFactor=1.1,
                    minNeighbors=10,
                    minSize=(80, 80)
                )

                for i, (x, y, w, h) in enumerate(faces):
                    face = img[y:y + h, x:x + w]  # Recortar rostro
                    face_resized = cv2.resize(face, img_size)  # Redimensionar
                    face_cutted = face_resized[
                        y_inicial:y_inicial + alto,
                        x_inicial:x_inicial + ancho
                    ]

                    # Crear carpeta de salida manteniendo la estructura original
                    relative_path = os.path.relpath(carpeta_actual, ruta_entrada)
                    output_folder = os.path.join(ruta_salida, relative_path)
                    os.makedirs(output_folder, exist_ok=True)

                    # Guardar rostro detectado
                    output_path = os.path.join(
                        output_folder,
                        f"{os.path.splitext(archivo)[0]}_face{i}.jpg"
                    )
                    cv2.imwrite(output_path, face_cutted)
                    print(f"✅ Guardado: {output_path}")

            except Exception as e:
                print(f"❌ Error procesando {ruta_completa}: {e}")

print("Detectar Caras ...")
procesar_archivos_en_carpetas(ruta_entrada, ruta_salida, img_size)
print("Detección y ajuste de tamaño terminado ...")